In [8]:
import pandas as pd
import networkx as nx

In [5]:
train_df =  pd.read_csv('../lm-vol/LANL_train_NTLM_Pikachu_filter.csv', dtype='str')
test_df =  pd.read_csv('../lm-vol/LANL_test_NTLM_Pikachu_filter.csv', dtype='str')

In [6]:
train_df.shape
test_df.shape

(3590236, 12)

In [16]:
graph_structure = ('src_user', 'dest_user')
G_test = nx.Graph()
for chunk in pd.read_csv('../lm-vol/LANL_train_NTLM_Pikachu_filter.csv', chunksize=1000000):
    for index, row in chunk.iterrows():
        user_node = row[graph_structure[0]]
        dest_node = row[graph_structure[1]]
        G_test.add_nodes_from([user_node, dest_node])
        G_test.add_edge(user_node, dest_node)

/tmp/ipykernel_382635/2206492509.py:3: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../lm-vol/LANL_train_NTLM_Pikachu_filter.csv', chunksize=1000000):


In [17]:
graph_structure = ('src_user', 'source computer', 'dest_user')
G_test_v2 = nx.Graph()
for chunk in pd.read_csv('../lm-vol/LANL_train_NTLM_Pikachu_filter.csv', chunksize=1000000):
    for index, row in chunk.iterrows():
        user_node = row[graph_structure[0]]
        ip_node = row[graph_structure[1]]
        dest_node = row[graph_structure[2]]
        G_test_v2.add_nodes_from([user_node, ip_node, dest_node])
        G_test_v2.add_edge(user_node, ip_node)
        G_test_v2.add_edge(ip_node, dest_node)

/tmp/ipykernel_382635/1988734659.py:3: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../lm-vol/LANL_train_NTLM_Pikachu_filter.csv', chunksize=1000000):


In [13]:
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

test_graph = StellarGraph.from_networkx(G_test)
test_graph_v2 = StellarGraph.from_networkx(G_test_v2)

In [14]:
print(test_graph.info())
print(test_graph_v2.info())

StellarGraph: Undirected multigraph
 Nodes: 7508, Edges: 7507

 Node types:
  default: [7508]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [7507]
        Weights: all 1 (default)
        Features: none
StellarGraph: Undirected multigraph
 Nodes: 17621, Edges: 18028

 Node types:
  default: [17621]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [18028]
        Weights: all 1 (default)
        Features: none


In [18]:
nx.write_graphml(G_test, "../lm-vol/LANL_train_NTLM_Pikachu_filter.graphml")
nx.write_graphml(G_test_v2, "../lm-vol/LANL_train_NTLM_Pikachu_filter_v2.graphml")

In [ ]:
import dask.dataframe as dd

In [ ]:
names = ['time','source user@domain','destination user@domain','source computer','destination computer','authentication type','logon type','authentication orientation','success/failure','day']
df =  dd.read_csv('../lm-vol/LANL_train_NTLM.csv', dtype='str', header= None, names=names, blocksize="1024MB")

In [ ]:
train_df =  dd.read_csv('../lm-vol/LANL_train.csv', dtype='str', header= None, names=names, blocksize="1024MB")

In [ ]:
df.head()

In [ ]:
df = df.assign(src_user = df['source user@domain'].str.split('@').map(lambda x: x[0]))
df = df.assign(dest_user = df['destination user@domain'].str.split('@').map(lambda x: x[0]))

In [ ]:
df.head()

In [ ]:
df = df[(df['src_user'].str.contains('LOCAL')==False) & (df['src_user'].str.contains('SYSTEM')==False) & (df['src_user'].str.contains('ANONYMOUS')==False) & (df['src_user'].str.contains('ADMINISTRARTOR')==False)]

In [ ]:
df = df[df['src_user'].str[-1] != '$']

In [ ]:
df.head()

In [ ]:
df = df[(df['dest_user'].str.contains('LOCAL')==False) & (df['dest_user'].str.contains('SYSTEM')==False) & (df['dest_user'].str.contains('ANONYMOUS')==False) & (df['dest_user'].str.contains('ADMINISTRARTOR')==False)]
df = df[df['dest_user'].str[-1] != '$']

In [ ]:
df.head()

In [ ]:
df.shape[0].compute()

In [ ]:
df.to_csv("../lm-vol/LANL_train_NTLM_Pikachu_filter.csv", index=False, single_file=True)

In [ ]:
df.to_parquet("../lm-vol/LANL_test_Pickachu_filter.parquet", engine='pyarrow')

In [ ]:
ntlm_df = df[df['authentication type'] == 'Kerberos']
ntlm_df.head()

In [ ]:
ntlm_df.to_csv('../lm-vol/LANL_test_Kerberos.csv', single_file=True)

In [ ]:
ntlm_df.shape[0].compute()

In [ ]:
ntlm_train_df = train_df[train_df['authentication type'] == 'Kerberos']
ntlm_train_df.head()

In [ ]:
ntlm_train_df.shape[0].compute()

In [ ]:
ntlm_train_df.to_csv('../lm-vol/LANL_train_Kerberos.csv', single_file=True)

In [ ]:
df2.shape[0].compute()

In [ ]:
train_df2.shape[0].compute()

In [ ]:
df2 = df2.repartition(npartitions=15)
df2.npartitions

In [ ]:
train_df2.npartitions

In [ ]:
df2 = df2.persist()

In [ ]:
max = 100
for index, row in df2.iterrows():
    if index > max:
        break
    print(row)

In [ ]:
display(df2)

In [ ]:
import multiprocessing
import numpy as np
import pandas as pd
import pickle
import gensim
from gensim.models import KeyedVectors

In [ ]:
for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
    NUM_CORES = 20
    # split the raw dataframe into chunks
    df_chunks = np.array_split(chunk ,NUM_CORES)
    # use a pool to spawn multiple proecsses
    with multiprocessing.Pool(NUM_CORES) as pool:
        # concatenate all processed chunks together.
        # process_df_function is the function you defined in the previous block
        processed_df = pd.concat(pool.map(process_df_function, df_chunks), ignore_index=True)
    processed_anomalies = processed_df[(processed_df["ip_to_service_pred"].astype(float) <= 0.1) | (processed_df["client_to_ip_pred"].astype(float) <= 0.1)]